In [1]:
import pandas as pd
import re
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import ast
import spacy

nltk.download('stopwords')
nltk.download('wordnet')
stemmer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
target = pd.read_csv('data/korea_herald_2015_30_ver_preprocessing.csv')


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

updated_cluster = [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29]]
cluster_vector = []
cluster_feature = []

for i in updated_cluster:
    cluster = target[target['cluster'].isin(i)]
    vectorizer = CountVectorizer(max_features=1500, ngram_range=(3, 6), min_df=5, max_df=0.6, stop_words=stopwords.words('english'))
    X_count = vectorizer.fit_transform(cluster.body).toarray()
    cluster_vector.append(X_count)
    cluster_feature.append(vectorizer.get_feature_names())

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
cluster_vector_tfidf = [TfidfTransformer().fit_transform(x).toarray() for x in cluster_vector]
df_tfidf = [pd.DataFrame(cluster_vector_tfidf[i], columns=cluster_feature[i]) for i in range(len(updated_cluster))]

In [9]:
def preprocessing(rawtext):
    document = rawtext
    document = re.sub(r'said', '', document)

    dates = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'week', 'next', 'month', 'year']
    for date in dates:
        document = re.sub(r'{}'.format(date), '', document)
    
    document = re.sub(r'\s+', ' ', document)
    return document
    
def remove_duplicates(candidates):
    result = []
    for cand in candidates:
        is_duplicate = False
        for cand2 in candidates:
            if cand == cand2:
                continue
            if cand in cand2:
                is_duplicate = True
                break
        if not is_duplicate:
            result.append(cand)
    return result
    

In [10]:
n_cluster = 15

def remove_duplicates(candidates):
    result = []
    for cand in candidates:
        is_duplicate = False
        for cand2 in candidates:
            if cand == cand2:
                continue
            if cand in cand2:
                is_duplicate = True
                break
        if not is_duplicate:
            result.append(cand)
    return result

for i in range(len(updated_cluster)):
    # remove word 'cluster'
    tf_idf_ne = df_tfidf[i].sum(axis=0).sort_values(ascending=False).keys()[:100]
    print("==============topic candidate: {}==============".format(updated_cluster[i]))
    print(remove_duplicates(list(tf_idf_ne))[:15])
    # break
    # print(list(tf_idf))
print("=============================================")

==============topic candidate: [0]==============
['president park geun hye', 'south korea japan', 'world war ii', 'cheong wa dae', 'foreign ministry said', 'foreign minister yun byung se', 'south korea china', 'said south korea', 'south north korea', 'president barack obama', 'north korea nuclear', '1950 53 korean war', 'japanese prime minister shinzo abe', 'ruling saenuri party', 'middle east respiratory syndrome']
==============topic candidate: [1]==============
['president park geun hye', 'south korean military', 'south korea united states', 'joint chiefs staff', 'north korean leader kim jong un', 'korean central news agency', 'inter korean relations', 'north korea nuclear', 'high level talks', 'said north korea', 'south north korea', 'inter korean ties', 'joint military drills', 'long range rocket', 'south korean firms']
==============topic candidate: [2]==============
['finance ministry said', 'president park geun hye', 'percent year earlier', 'government said wednesday', 'ruling 